# Jupyter notebook version of the line example

This example shows how to run and document a model experiment with LADiM in a jupyter notebook.

The example is modified from a notebook from Pål N. Sævik

In [ ]:
import numpy as np
import ladim

In [ ]:
# Main settings
start_time = '1989-05-24T12:00:00'
stop_time  = '1989-06-15T12:00:00'
input_file    = '../data/ocean_avg_0014.nc'
particle_file = 'line.nc'

## Release file

First make a release file, initializing a line of particles

In [ ]:
rls_file = 'line.rls'

# End points of the line in grid coordinates
x0, x1 = 63.55, 123.45
y0, y1 = 90.0, 90.0

# Generate the line
Npart = 1000
X = np.linspace(x0, x1, Npart)
Y = np.linspace(y0, y1, Npart)

# Particle depth
Z = 5 

with open(rls_file, mode='w', encoding='utf-8') as fid:
    for i, (x, y) in enumerate(zip(X, Y)):
        fid.write(f'{start_time:s} {x:7.3f} {y:7.3f} {Z:6.1f}\n')

## Config file

This file configures the numerical experiment

In [ ]:
yaml_file = 'ladim.yaml'
yaml_text = f"""
time_control:
    # Start and stop of simulation
    start_time : {start_time}
    stop_time  : {stop_time}

files:
    grid_file             : {input_file}
    input_file            : {input_file}
    particle_release_file : {rls_file}
    output_file           : {particle_file}

state:
    # pid, X, Y, Z are mandatory and should not be given
    # Ingen, kan bruke [] eller kutte linjen
    # ibm_variables: []

particle_release:
    variables: [release_time, X, Y, Z]
    # Converters (if not float)
    release_time: time
    particle_variables: [release_time]

gridforce:
    module: ladim.gridforce.ROMS

output_variables:
    # Output period, format [value, unit], unit = s, m, h, or d
    outper: [3, h]
    # Variable names
    particle: [release_time]
    instance: [pid, X, Y, Z]
    # NetCDF arguments
    release_time:
        ncformat: f8
        long_name: particle release time
        units: seconds since reference_time
    pid: {{ncformat: i4, long_name: particle identifier}}
    X: {{ncformat: f4, long_name: particle X-coordinate}}
    Y: {{ncformat: f4, long_name: particle Y-coordinate}}
    Z:
        ncformat: f4
        long_name: particle depth
        standard_name: depth_below_surface
        units: m
        positive: down

numerics:
    # Model time step, [value, unit]
    dt: [1, h]
    # Advection method: options =
    #        EF = Euler-Forward,
    #        RK2, RK4 = Runge-Kutta 2nd or 4th order
    advection: RK4
    # Horizontal diffusion coefficient [m2.s-1]
    #         zero = no diffusion
    diffusion: 0.0
"""

with open(yaml_file, 'w', encoding='utf-8') as fid:
    fid.write(yaml_text)

# Ladim simulation

In [ ]:
# import ladim
# import logging
# ladim.main(loglevel=logging.WARNING)
ladim.main(config_file='ladim.yaml', loglevel=20)

# Plot moving particles

In [ ]:
from postladim import ParticleFile
import matplotlib.pyplot as plt
from netCDF4 import Dataset



# Subgrid for plotting
i0, i1 = 50, 150
j0, j1 = 60, 140

with Dataset(input_file) as ncid:
    H = ncid.variables['h'][j0:j1, i0:i1]
    M = ncid.variables['mask_rho'][j0:j1, i0:i1]
# X, Y = pf.position(0)

# pf = ParticleFile(particle_file)

# fig = plt.figure()
# dots = plt.plot(X, Y, 'ro')[0]
# plt.xlim([4.9, 5.0])
# plt.ylim([60.52, 60.56])

# Cell centers and boundaries
Xcell = np.arange(i0, i1)
Ycell = np.arange(j0, j1)
Xb = np.arange(i0-0.5, i1)
Yb = np.arange(j0-0.5, j1)

# particle_file
pf = ParticleFile(particle_file)
num_times = pf.num_times

# Set up the plot area
fig = plt.figure(figsize=(8, 6))
ax = plt.axes(xlim=(i0+1, i1-1), ylim=(j0+1, j1-1), aspect='equal')

# Background bathymetry
cmap = plt.get_cmap('Blues')
ax.contourf(Xcell, Ycell, H, cmap=cmap, alpha=0.6)

# Lon/lat lines
#ax.contour(Xcell, Ycell, lat, levels=range(57, 64),
#           colors='black', linestyles=':')
#ax.contour(Xcell, Ycell, lon, levels=range(-4, 10, 2),
#           colors='black', linestyles=':')

# Landmask
constmap = plt.matplotlib.colors.ListedColormap([0.2, 0.6, 0.4])
M = np.ma.masked_where(M > 0, M)
ax.pcolormesh(Xb, Yb, M, cmap=constmap)

dots, = ax.plot(X, Y, '.', color='red', markeredgewidth=0, lw=2.5)


def plot_dots(timestep):
    X, Y = pf.position(timestep)
    dots.set_data(X, Y)
    return dots

plot_dots(0);

In [ ]:
import matplotlib.animation as animation
import matplotlib
from IPython.display import HTML

anim = animation.FuncAnimation(fig, plot_dots,
    frames=pf.num_times, interval=50, repeat=False)

HTML(anim.to_html5_video())